# Exercise 3: Spatial architecture of tumor microenvironment with SCIMAP

**Objective:** Use Scimap to determine whether immune cells are successfully infiltrating the tumor — or getting "trapped" at the interface.

---

## Clinical Context

This exercise uses data from a study of **high-grade serous ovarian cancer (HGSOC)** — one of the most lethal gynaecological cancers, largely because most patients eventually develop resistance to standard platinum-based chemotherapy.

Two major treatment strategies exist, and they attract very different immune landscapes:

| Strategy | Abbreviation | Description |
|----------|-------------|-------------|
| Primary Debulking Surgery | **PDS** | Surgery first, then chemotherapy |
| Interval Debulking Surgery | **IDS** | Chemotherapy first to shrink the tumour, then surgery |

Patients are also stratified by their **homologous recombination (HR) status** — a measure of DNA repair capacity:

- **BRCALoss** — BRCA1/2 mutations impair DNA repair, making tumours more sensitive to certain therapies and more immunogenic
- **HRP** (HR-proficient) — intact DNA repair, generally associated with poorer immunotherapy response

In the study ([Perez-Villatoro et al., 2026](https://doi.org/10.1158/2159-8290.CD-25-1492)), different treatment–genotype combinations produced strikingly different **tumour–stroma interfaces (TSI)** — the boundary zone where immune cells either successfully invade the tumour or get blocked.

---

## What You Will Discover

We will compare two TMA cores that represent opposite ends of the immune infiltration spectrum:

| Core | Patient Group |
|------|--------------|
| **Core 84** | BRCALoss + PDS |
| **Core 42** | HRP + IDS |



> **Key question:** Can you detect the differences in spatial architecture of tumor microenvironment computationally — using only the spatial coordinates and protein expression of individual cells?

---

## Learning outcomes

By the end of this exercise you will be able to:

1. Visualise raw marker expression across a tissue section.
2. Build a rule-based cell phenotyping workflow.
3. Quantify the physical proximity of immune cells to tumour cells.
4. Identify **Recurrent Cell Neighbourhoods (RCNs)** and interpret their biological meaning.

---

## Workflow Overview

```
Step 1 → Load Data
Step 2 → Visualise Raw Markers
Step 3 → Phenotype Cells
Step 4 → Quality Control
Step 5 → Measure Spatial Distances
Step 6 → Map Cell Neighbourhoods
```

## Step 1: Load the Data

We start by importing our libraries and loading the dataset. The H5AD format (used by AnnData/Scanpy) stores:
- **`adata.X`** — the protein expression matrix (cells × markers)
- **`adata.obs`** — per-cell metadata (image ID, coordinates, phenotype labels)
- **`adata.obsm['spatial']`** — the X, Y pixel coordinates of each cell in the tissue image

After loading, we split the combined dataset into the two TMA cores for independent processing.

In [ ]:
import scimap as sm
import scanpy as sc
import anndata as ad
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Load the combined dataset
adata = sc.read_h5ad('single_cell_TMA_dataset.h5ad')

# Split into the two TMA cores for independent processing
tma8 = adata[adata.obs['imageid'] == 'TMA_8_core84'].copy()
tma1 = adata[adata.obs['imageid'] == 'TMA_1_core42'].copy()

print(f"Loaded {tma8.n_obs:,} cells for Core 84 and {tma1.n_obs:,} cells for Core 42.")
print(f"\nDataset schema:")
print(f"  Markers : {list(adata.var_names)}")
print(f"  Metadata columns : {list(adata.obs.columns)}")
print(f"  Spatial coordinates stored in : adata.obsm['spatial']")

## Step 2: Visualise Raw Marker Expression

Before calculating anything, we need to build an intuition for the data. We create a **Marker Atlas** — a spatial map of each protein's expression across the tissue — for both cores side by side.

This serves two purposes:
1. It lets us visually confirm which regions of the tissue express which markers.
2. It guides us in writing the phenotyping rules in the next step.

> Generating these plots may take a few seconds.

In [ ]:
# Define the markers of interest for this exercise
marker_cols = ['CK7', 'ECadherin', 'aSMA', 'Vimentin', 'CD3d', 'CD11c']

# Custom blue-teal-dark colormap for expression intensity
custom_cmap = mcolors.LinearSegmentedColormap.from_list(
    "white_teal_dark", ["#FFFFFF", "#508791", "#1D3033"]
)

# --- Marker Atlas: Core 84 ---
print("Generating Marker Atlas for Core 84...")
sm.pl.spatial_scatterPlot(
    tma8, colorBy=marker_cols, s=3, ncols=2,
    figsize=(10, 12), cmap=custom_cmap, plotLegend=True
)

# --- Marker Atlas: Core 42 ---
print("\nGenerating Marker Atlas for Core 42...")
sm.pl.spatial_scatterPlot(
    tma1, colorBy=marker_cols, s=3, ncols=2,
    figsize=(10, 12), cmap=custom_cmap, plotLegend=True
)

### Marker Reference Table

| Marker | Cell Type | Biological Role |
|--------|-----------|-----------------|
| **CK7** | Epithelial / Cancer | Cytokeratin filament protein that delineates primary tumour nests |
| **E-Cadherin** | Epithelial / Cancer | Calcium-dependent cell–cell adhesion molecule maintaining epithelial integrity |
| **αSMA** | Stromal / Fibroblast | Actin isoform expressed in activated cancer-associated fibroblasts |
| **Vimentin** | Stromal / Mesenchymal | Intermediate filament broadly marking non-epithelial (mesenchymal) cells |
| **CD3d** | Lymphoid / T cell | Core component of the T-cell receptor signalling complex |
| **CD11c** | Myeloid / Dendritic cell | Integrin expressed at high levels on myeloid cells (DCs, macrophages) |

:::{admonition} QUESTIONS:
:class: tip

Look at the Marker Atlas for both cores and consider:

1. Which markers show clearly distinct spatial boundaries, and which appear diffuse?
2. Based on raw intensity alone, which markers do you expect will be **easy** vs **challenging** to threshold into positives/negatives? Why?
3. Do the two cores look similar or different in terms of marker distribution? What does this hint at biologically?
:::


## Step 3: Define Cell Identities

Now that we've seen the raw markers, we need to teach the algorithm how to recognise each cell type. We use a **rule-based phenotyping** approach, where each cell type is defined by the presence (`pos`) or absence (`NaN`) of specific markers.

### The Rules

| Phenotype | Required Markers |
|-----------|-----------------|
| **Cancer** | CK7 **AND** ECadherin must be positive |
| **Stromal** | αSMA **AND** Vimentin must be positive |
| **Myeloid** | CD11c must be positive |
| **Lymphoid** | CD3d must be positive |

:::{admonition} **Task:** 
Fill in the `None` values in the `data` list below with `'pos'` where a marker must be present for each phenotype. Leave `None` where the marker should be absent or irrelevant.
:::

In [ ]:
# Column order: Global | Phenotype | CK7 | ECadherin | aSMA | Vimentin | CD11c | CD3d
columns = ['Global', 'Phenotype', 'CK7', 'ECadherin', 'aSMA', 'Vimentin', 'CD11c', 'CD3d']

# TODO: Replace None with 'pos' where a marker must be expressed for that phenotype
data = [
    #  Global    Phenotype    CK7    ECad   aSMA   Vim    CD11c  CD3d
    ['all',    'Cancer',    None,  None,  None,  None,  None,  None],
    ['all',    'Stromal',   None,  None,  None,  None,  None,  None],
    ['all',    'Myeloid',   None,  None,  None,  None,  None,  None],
    ['all',    'Lymphoid',  None,  None,  None,  None,  None,  None],
]

# Build DataFrame and save to CSV — this is the input format Scimap expects
phenotype_workflow = pd.DataFrame(data, columns=columns)
print("Phenotyping workflow:")
print(phenotype_workflow.to_string(index=False))

# Save the workflow for reuse
phenotype_workflow.to_csv('phenotype_workflow.csv', index=False)

### Run the Classification

With our rules defined, we can now apply them. Scimap first **rescales** each marker's intensity within [0, 1] per image (to correct for batch effects between cores), then applies the phenotyping logic.

In [ ]:
# Rescale marker intensities per image to correct for batch effects
# This maps each marker's distribution to [0, 1] so thresholds are comparable
tma8 = sm.pp.rescale(tma8, gate=None)
tma1 = sm.pp.rescale(tma1, gate=None)
print("Rescaling complete.")

In [ ]:
# Step 2: Apply the phenotyping rules to classify each cell
phenotype_workflow = pd.read_csv('phenotype_workflow.csv')

tma8 = sm.tl.phenotype_cells(tma8, phenotype=phenotype_workflow, label='phenotype')
tma1 = sm.tl.phenotype_cells(tma1, phenotype=phenotype_workflow, label='phenotype')

# Checkpoint: How many cells were assigned to each phenotype?
print("=== Core 84 — Phenotype Counts ===")
print(tma8.obs['phenotype'].value_counts().to_string())
print("\n=== Core 42 — Phenotype Counts ===")
print(tma1.obs['phenotype'].value_counts().to_string())

## Step 4: Quality Control

**Never skip QC.** After automated phenotyping, we must verify the results make biological sense. We use two complementary checks:

**A) Marker heatmap** — Each row is a phenotype; each column is a marker. If the logic is correct:
- The `Cancer` row should show high expression **only** for CK7 and ECadherin
- The `Lymphoid` row should show high expression **only** for CD3d
- If a phenotype row lights up for unexpected markers, your rules need refinement

**B) Spatial scatter plot** — Overlay the phenotype labels back onto the tissue to check that the spatial distribution makes anatomical sense (e.g., cancer cells should form coherent nests, not scatter randomly).

In [ ]:
# QC A: Marker expression heatmap — Core 84
print("Heatmap for Core 84:")
sm.pl.heatmap(tma8, groupBy='phenotype', subsetMarkers=marker_cols, figsize=(8, 5))

In [ ]:
# QC A: Marker expression heatmap — Core 42
print("Heatmap for Core 42:")
sm.pl.heatmap(tma1, groupBy='phenotype', subsetMarkers=marker_cols, figsize=(8, 5))

In [ ]:
# QC B: Spatial scatter plot of phenotypes — Core 84
print("Spatial phenotype map for Core 84:")
sm.pl.spatial_scatterPlot(
    tma8, colorBy='phenotype',
    figsize=(6, 5), plotLegend=True
)

In [ ]:
# QC B: Spatial scatter plot of phenotypes — Core 42
print("Spatial phenotype map for Core 42:")
sm.pl.spatial_scatterPlot(
    tma1, colorBy='phenotype',
    figsize=(6, 5), plotLegend=True
)

:::{admonition} QUESTIONS:
:class: tip

Examine the heatmaps and spatial plots above:

1. Do any phenotype rows in the heatmap show unexpectedly high expression for off-target markers? What might cause this?
2. Which phenotype is **most sensitive** to the threshold choice, and why? *(Hint: think about markers that are expressed broadly at low levels.)*
3. Do the "Cancer" cells form coherent spatial nests in the tissue plot, or are they scattered? What does that tell you about the classification quality?
:::


In [ ]:
# Clean up: Merge low-confidence "likely-" subcategories into 'Unknown'
# These are cells that partially match a phenotype but don't meet all criteria
for tma in [tma8, tma1]:
    tma.obs['phenotype'] = tma.obs['phenotype'].replace({
        'likely-Cancer': 'Unknown',
        'likely-Stromal': 'Unknown'
    })

print("Low-confidence cells re-labelled as 'Unknown'.")
print("\nUpdated counts — Core 84:")
print(tma8.obs['phenotype'].value_counts().to_string())
print("\nUpdated counts — Core 42:")
print(tma1.obs['phenotype'].value_counts().to_string())

## Step 5: Measuring the "Social Distance" Between Cell Types

Now we get to the heart of the exercise. We will measure the **physical distance** from every Cancer cell to its nearest neighbour of each other phenotype.

### Why does this matter?

- A **left-shifted** distance distribution (peak at short distances) means immune cells are **physically close** to tumour cells — consistent with active infiltration.
- A **right-shifted** distribution (peak at longer distances) suggests immune cells are present in the tissue but **excluded** from the tumour.

:::{admonition} **Before running the code:**
Predict which core you think will show greater immune–tumour proximity, based on the spatial maps you examined in Step 4.
:::

In [ ]:
# Define a consistent colour palette for cell types across all plots
celltype_colors = {
    'Cancer':   '#D33F49',  # Red
    'Stromal':  '#FED18C',  # Peach
    'Lymphoid': '#6599CD',  # Blue
    'Myeloid':  '#508791',  # Teal
    'Unknown':  '#D9DEE4',  # Grey
}

# Combine both cores into one AnnData object for side-by-side comparison
adata = ad.concat(
    [tma8, tma1], join='outer',
    label='batch', keys=['core84', 'core42']
)

# Calculate pairwise spatial distances between all phenotype combinations
adata = sm.tl.spatial_distance(adata, phenotype='phenotype')

print("Spatial distances calculated. Summary:")
print(f"  Total cells: {adata.n_obs:,}")
print(f"  Phenotypes: {list(adata.obs['phenotype'].unique())}")

In [ ]:
# Plot A: Numeric distance summary — median distance from Cancer to each cell type
# This gives a quick overview of the 'social landscape' around tumour cells
sm.pl.spatial_distance(
    adata, method='numeric',
    distance_from='Cancer',
    log=True, height=3, aspect=11/8,
    palette=celltype_colors
)

In [ ]:
# Plot B: Distribution of Cancer → Lymphoid distances, comparing both cores
# This is the key diagnostic plot for immune infiltration vs exclusion
sm.pl.spatial_distance(
    adata, method='distribution',
    distance_from='Cancer', distance_to='Lymphoid',
    log=True, height=4, aspect=1.5,
    palette={'TMA_8_core84': '#FED18C', 'TMA_1_core42': '#A7AEF2'}
)

:::{admonition} TASK
:class: tip

Examine the Cancer → Lymphoid distance distribution above and answer:

1. **The "Close Contact" Peak:** Which core has a higher density of Lymphoid cells at very short distances from Cancer cells?
2. **The "Exclusion" Gap:** Does either core show a pronounced hump at a *longer* distance? This suggests immune cells are present in the tissue but physically barred from entering the tumour.
3. **Clinical Classification:** In immunotherapy research, tumours are classified as *Inflamed* (T cells inside tumour), *Excluded* (T cells at border), or *Desert* (no T cells). Based only on this plot, how would you classify each core?

> Did your prediction match the result? If not, what might explain the difference?
:::


## Step 6: Mapping Recurrent Cell Neighbourhoods (RCNs)

Distance analysis tells us *how far* cells are from each other. But to understand the full tissue architecture, we need to identify distinct **Recurrent Cell Neighbourhoods (RCNs)** — recurrent spatial patterns where specific combinations of cell types co-occur.

### The RCN approach in three steps:
1. For each cell, count how many neighbours of each phenotype live within a radius *r*
2. Use **k-means clustering** on these neighbourhood profiles to group cells with similar local environments
3. Assign biological names to each cluster based on its dominant cell types

:::{admonition} Predict before you compute

Before running the code, think: **what distinct neighbourhoods would you expect to find in a tumour microenvironment?** For example:
- Pure tumour cores?
- Stromal regions?
- Immune-rich zones?
- Mixed interface regions?

Write down your predictions, then compare them to the clusters you discover.
:::


In [ ]:
# Remove cells with ambiguous phenotype for cleaner neighbourhood analysis
adata_known = adata[adata.obs['phenotype'] != 'Unknown'].copy()

print(f"Cells retained after removing Unknowns: {adata_known.n_obs:,}")
print(f"Phenotype breakdown:")
print(adata_known.obs['phenotype'].value_counts().to_string())

In [ ]:
# For every cell: count neighbours of each phenotype within a 50-pixel radius
# This radius is a biological decision — too small = noisy, too large = loses local structure
sm.tl.spatial_count(adata_known, method='radius', radius=50, label='spatial_count')

print("Neighbourhood counts computed. Each cell now has a 'neighbourhood profile'.")

### Finding the Optimal Number of Neighbourhoods

We use the **elbow method** to choose how many RCNs to define. We plot the k-means inertia (sum of squared distances to cluster centres) for different values of *k*. The "elbow" — where the inertia starts decreasing much more slowly — is a good choice for *k*.

> **What to look for:** Find the value of *k* where the curve bends most sharply. Adding clusters beyond that point gives diminishing returns.

In [ ]:
from sklearn.cluster import KMeans

# Extract the neighbourhood count matrix
neighbor_data = adata_known.uns['spatial_count']

# Run k-means for k = 1 to 10, recording inertia at each step
inertia = []
K_range = range(1, 11)

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(neighbor_data)
    inertia.append(km.inertia_)

# Plot the elbow curve
fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(K_range, inertia, 'o-', color='#508791', linewidth=2, markersize=8)
ax.set_xlabel('Number of Clusters (k)', fontsize=13)
ax.set_ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=13)
ax.set_title('Elbow Method — Choosing the Number of RCNs', fontsize=14)
ax.axvline(x=5, color='#D33F49', linestyle='--', alpha=0.7, label='Chosen k = 5')
ax.legend()
plt.tight_layout()
plt.show()

print("\nBased on the elbow curve, k=5 is a reasonable choice.")

In [ ]:
# Cluster cells into 5 Regional Cell Neighbourhoods
# Clustering both cores together ensures cluster labels are comparable across images
sm.tl.spatial_cluster(adata_known, label='spatial_cluster', k=5, method='kmeans')

print("Cells assigned to 5 spatial clusters.")
print(adata_known.obs['spatial_cluster'].value_counts().to_string())

### Interpreting the Neighbourhoods

Now we inspect the cell-type composition of each cluster to assign biological names. We use a stacked bar plot — the dominant phenotype in each cluster determines its identity.

In [ ]:
# Generate stacked barplots for each core and retrieve the underlying data

import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    df_84 = sm.pl.stacked_barplot(
        adata_known[adata_known.obs['imageid'] == 'TMA_8_core84'],
        x_axis='spatial_cluster', y_axis='phenotype', return_data=True,
        saveDir=tmpdir, fileName='tmp.pdf'
    )

    df_42 = sm.pl.stacked_barplot(
        adata_known[adata_known.obs['imageid'] == 'TMA_1_core42'],
        x_axis='spatial_cluster', y_axis='phenotype', return_data=True,
        saveDir=tmpdir, fileName='tmp.pdf'
    )

# Manually plot with the biological colour palette for consistency
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

df_84.plot(kind='bar', stacked=True, color=celltype_colors, ax=ax1, legend=False)
ax1.set_title('Core 84: Neighbourhood Composition', fontsize=13)
ax1.set_ylabel('Proportion of Cells', fontsize=12)
ax1.set_xlabel('Spatial Cluster', fontsize=12)
ax1.tick_params(axis='x', rotation=0)

df_42.plot(kind='bar', stacked=True, color=celltype_colors, ax=ax2)
ax2.set_title('Core 42: Neighbourhood Composition', fontsize=13)
ax2.set_xlabel('Spatial Cluster', fontsize=12)
ax2.tick_params(axis='x', rotation=0)
ax2.legend(title='Phenotype', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.suptitle('Regional Cell Neighbourhood Composition by Core', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Print the raw proportions for detailed inspection
print("=== Core 84 — Cluster Compositions ===")
print(df_84.round(3).to_string())
print("\n=== Core 42 — Cluster Compositions ===")
print(df_42.round(3).to_string())

### Assign Biological Names to Clusters

Based on the bar plots above, assign a biological name to each cluster. The template below uses example names — **update the `rcn_names` dictionary to match what you observe in your own results**.

In [ ]:
# Update these IDs based on your actual stacked barplot output above
# The key is the cluster number (as a string); the value is the biological name
rcn_names = {
    '0': 'Myeloid-Rich Niche',
    '1': 'Stroma',
    '2': 'Solid Tumour Core',
    '3': 'Lymphoid-Rich Niche',
    '4': 'Tumour-Stromal Interface'
}

# Add a new column with readable names, preserving the numeric cluster IDs
adata_known.obs['spatial_community'] = (
    adata_known.obs['spatial_cluster']
    .astype(str)
    .replace(rcn_names)
)

# Define a palette for the community names
community_palette = {
    'Tumour-Stromal Interface': '#508791',  # Teal
    'Solid Tumour Core':        '#DC19DC',  # Magenta
    'Stroma':                   '#FED18C',  # Peach
    'Myeloid-Rich Niche':       '#FA3232',  # Red
    'Lymphoid-Rich Niche':      '#A7AEF2',  # Lavender
}

print("Communities assigned:", list(adata_known.obs['spatial_community'].unique()))

In [ ]:
# Final spatial maps coloured by Regional Cell Neighbourhood
print("RCN map — Core 84:")
sm.pl.spatial_scatterPlot(
    adata_known[adata_known.obs['imageid'] == 'TMA_8_core84'],
    colorBy='spatial_community', imageid='imageid',
    s=12, customColors=community_palette, figsize=(8, 5)
)

print("\nRCN map — Core 42:")
sm.pl.spatial_scatterPlot(
    adata_known[adata_known.obs['imageid'] == 'TMA_1_core42'],
    colorBy='spatial_community', imageid='imageid',
    s=12, customColors=community_palette, figsize=(8, 5)
)

:::{admonition} TASK:
:class: tip

Examine the RCN spatial maps and consider:

1. **Core comparison:** Which RCN is proportionally larger in Core 84 vs Core 42? How does this relate to the distance analysis in Step 5?
2. **Radius sensitivity:** If you **doubled** the neighbourhood radius from 50 to 100 pixels, what would happen to the RCNs? Which biological structures would become *more* visible, and which would become *less specific*?
:::


## Summary & Conclusions

In this exercise you applied a complete spatial proteomics analysis pipeline to two TMA cores:

| Step | What you did | Key tool |
|------|--------------|----------|
| 1 | Loaded and split the AnnData dataset | `sc.read_h5ad` |
| 2 | Visualised raw marker expression spatially | `sm.pl.spatial_scatterPlot` |
| 3 | Built a rule-based cell phenotyping workflow | `sm.tl.phenotype_cells` |
| 4 | Validated phenotypes with heatmaps and spatial plots | `sm.pl.heatmap` |
| 5 | Quantified immune–tumour proximity distributions | `sm.tl.spatial_distance` |
| 6 | Discovered and named Recurrent Cell Neighbourhoods | `sm.tl.spatial_count`, `sm.tl.spatial_cluster` |

### Key Takeaways

- **Spatial context is everything.** The same number of immune cells can have very different clinical relevance depending on *where* they are relative to the tumour.
- **Rule-based phenotyping requires validation.** Always sanity-check your classifications with heatmaps and spatial overlays before moving to downstream analysis.
- **RCNs are scale-dependent.** The radius parameter is a biological decision, not just a technical one — it determines the spatial scale at which you define "neighbourhood."

---

## Going Further

Interested in extending this analysis? Here are some follow-up resources:

- [Scimap documentation](https://scimap.xyz)
- [AnnData documentation](https://anndata.readthedocs.io)